In [1]:
import duckdb

In [ ]:
con = duckdb.connect("crashes.duckdb")

In [5]:
con = duckdb.connect("crashes.duckdb")

# Create a table inside DuckDB from the Parquet file
con.execute("""
    CREATE OR REPLACE TABLE crashes AS 
    SELECT 
        BOROUGH,
        DAYNAME("CRASH DATE") AS WEEKDAY,
        EXTRACT(YEAR FROM "CRASH DATE") AS YEAR,
        COALESCE("NUMBER OF PERSONS KILLED", 0) AS PERSONS_KILLED,
        COALESCE("NUMBER OF PERSONS INJURED", 0) AS PERSONS_INJURED,
        "CONTRIBUTING FACTOR VEHICLE 1",
        "CONTRIBUTING FACTOR VEHICLE 2",
        "CONTRIBUTING FACTOR VEHICLE 3",
        "CONTRIBUTING FACTOR VEHICLE 4",
        "CONTRIBUTING FACTOR VEHICLE 5"
    FROM 'crashes.parquet'
""")

In [ ]:
print(con.execute("SELECT COUNT(*) FROM crashes").fetchall())

In [ ]:
# create a persistent connection to the DuckDB file
# con = duckdb.connect("crashes.duckdb", read_only=True)

In [8]:
df = con.execute("SELECT * FROM crashes").df()
df.head()

,BOROUGH,WEEKDAY,YEAR,PERSONS_KILLED,PERSONS_INJURED,CONTRIBUTING FACTOR VEHICLE 1,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5
0,BROOKLYN,Wednesday,2023,0.0,1.0,Unspecified,Unspecified,Unspecified,None,None
1,BROOKLYN,Saturday,2021,0.0,0.0,Unspecified,None,None,None,None
2,BROOKLYN,Tuesday,2021,0.0,0.0,None,None,None,None,None
3,BROOKLYN,Tuesday,2021,0.0,0.0,Passing Too Closely,Unspecified,None,None,None
4,BRONX,Tuesday,2021,0.0,2.0,Unspecified,Unspecified,None,None,None


In [ ]:
df.shape

#### day

In [9]:
df = con.execute("""
    SELECT WEEKDAY, COUNT(WEEKDAY) AS counts
    FROM crashes
    GROUP BY WEEKDAY
    ORDER BY counts DESC;
 """).df()

df.head(7)

,WEEKDAY,counts
0,Friday,316505
1,Thursday,296651
2,Tuesday,292430
3,Wednesday,290171
4,Monday,284488
5,Saturday,268874
6,Sunday,239946


#### word

In [ ]:
df = con.execute("""
    WITH factors AS (
    SELECT "CONTRIBUTING FACTOR VEHICLE 1" AS reasons
    FROM crashes
    WHERE BOROUGH IN ? AND YEAR BETWEEN ? AND ?
    UNION ALL
    SELECT "CONTRIBUTING FACTOR VEHICLE 2"
    FROM crashes
    WHERE BOROUGH IN ? AND YEAR BETWEEN ? AND ?
    UNION ALL
    SELECT "CONTRIBUTING FACTOR VEHICLE 3"
    FROM crashes
    WHERE BOROUGH IN ? AND YEAR BETWEEN ? AND ?
    UNION ALL
    SELECT "CONTRIBUTING FACTOR VEHICLE 4"
    FROM crashes
    WHERE BOROUGH IN ? AND YEAR BETWEEN ? AND ?
    UNION ALL
    SELECT "CONTRIBUTING FACTOR VEHICLE 5"
    FROM crashes
    WHERE BOROUGH IN ? AND YEAR BETWEEN ? AND ?
)
SELECT reasons, COUNT(*) AS Count
FROM factors
WHERE reasons IS NOT NULL
GROUP BY reasons
ORDER BY Count DESC
LIMIT 25;
 """).df()

In [ ]:
df = con.execute("""
    WITH factors AS (
    SELECT Year, "CONTRIBUTING FACTOR VEHICLE 1" AS reasons
    FROM crashes
    UNION ALL
    SELECT Year, "CONTRIBUTING FACTOR VEHICLE 2"
    FROM crashes
    UNION ALL
    SELECT Year, "CONTRIBUTING FACTOR VEHICLE 3"
    FROM crashes
    UNION ALL
    SELECT Year, "CONTRIBUTING FACTOR VEHICLE 4"
    FROM crashes
    UNION ALL
    SELECT Year, "CONTRIBUTING FACTOR VEHICLE 5"
    FROM crashes
),
reasons AS (
    SELECT Year, reasons
    FROM factors
    WHERE YEAR BETWEEN 2020 AND 2021
),
words AS (
    SELECT reasons, COUNT(reasons) AS Count
    FROM reasons
    GROUP BY reasons
    ORDER BY Count DESC
) 
SELECT * 
FROM words
""").df()

In [ ]:
df.head(30)

In [ ]:
df.shape

#### kpi

In [ ]:
kpi_query = con.execute("""
    SELECT 
        COUNT(*) AS total_collisions, 
        SUM(PERSONS_KILLED) AS persons_killed, 
        SUM(PERSONS_INJURED) AS persons_injured
    FROM crashes
""").fetchone()

total_collisions, persons_killed, persons_injured = map(int, kpi_query)